In [12]:
import xgboost as xgb
from utils.mnist_reader import load_mnist
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize, StandardScaler
import random
from sklearn.ensemble import VotingClassifier

### Loading the fashion dataset...

In [2]:
X_main, y_main = load_mnist("data/fashion", kind="train")
X_main = X_main.astype(np.float32)
y_main = y_main.astype(np.float32)

X_test, y_test = load_mnist("data/fashion", kind="t10k")
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

print (X_main.shape, y_main.shape)
print (X_test.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


### Loading MNIST dataset...

In [28]:
X_mnist, y_mnist = load_mnist("mnist", kind="train")
X_mnist = X_mnist.astype(np.float32)
y_mnist = y_mnist.astype(np.float32)

X_test_mnist, y_test_mnist = load_mnist("mnist", kind="t10k")
X_test_mnist = X_test_mnist.astype(np.float32)
y_test_mnist = y_test_mnist.astype(np.float32)

print (X_mnist.shape, y_mnist.shape)
print (X_test_mnist.shape, y_test_mnist.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


### Standard scaling the pixel values with mean=0.0 and var=1.0

In [3]:
sc = StandardScaler()
X_main_std = sc.fit_transform(X_main)

In [29]:
X_mnist_std = sc.fit_transform(X_mnist)

### Splitting the train dataset into train and validation sets

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X_main_std, y_main, test_size=0.1)

In [30]:
X_mn_train, X_mn_valid, y_mn_train, y_mn_valid = train_test_split(X_mnist_std, y_mnist, test_size=0.1)

### Training XgBoost classifier on fashion-mnist dataset

In [35]:
param_list = [("eta", 0.08), ("max_depth", 6), ("subsample", 0.8), ("colsample_bytree", 0.8), ("objective", "multi:softmax"), ("eval_metric", "merror"), ("alpha", 8), ("lambda", 2), ("num_class", 10)]
n_rounds = 600
early_stopping = 50
    
d_train = xgb.DMatrix(X_train, label=y_train)
d_val = xgb.DMatrix(X_valid, label=y_valid)
eval_list = [(d_train, "train"), (d_val, "validation")]
bst = xgb.train(param_list, d_train, n_rounds, evals=eval_list, early_stopping_rounds=early_stopping, verbose_eval=True)

[0]	train-merror:0.175389	validation-merror:0.192333
Multiple eval metrics have been passed: 'validation-merror' will be used for early stopping.

Will train until validation-merror hasn't improved in 50 rounds.
[1]	train-merror:0.160148	validation-merror:0.178833
[2]	train-merror:0.154907	validation-merror:0.172167
[3]	train-merror:0.150019	validation-merror:0.168833
[4]	train-merror:0.146796	validation-merror:0.1625
[5]	train-merror:0.144741	validation-merror:0.161667
[6]	train-merror:0.142296	validation-merror:0.158333
[7]	train-merror:0.14113	validation-merror:0.158167
[8]	train-merror:0.138963	validation-merror:0.159167
[9]	train-merror:0.138241	validation-merror:0.158
[10]	train-merror:0.13687	validation-merror:0.155667
[11]	train-merror:0.135981	validation-merror:0.155333
[12]	train-merror:0.134926	validation-merror:0.153833
[13]	train-merror:0.134167	validation-merror:0.153333
[14]	train-merror:0.132722	validation-merror:0.152333
[15]	train-merror:0.131407	validation-merror:0.1

[151]	train-merror:0.050611	validation-merror:0.105667
[152]	train-merror:0.050222	validation-merror:0.105333
[153]	train-merror:0.049926	validation-merror:0.105167
[154]	train-merror:0.0495	validation-merror:0.104833
[155]	train-merror:0.049148	validation-merror:0.104167
[156]	train-merror:0.048796	validation-merror:0.1045
[157]	train-merror:0.0485	validation-merror:0.104333
[158]	train-merror:0.048148	validation-merror:0.104167
[159]	train-merror:0.047722	validation-merror:0.103667
[160]	train-merror:0.047352	validation-merror:0.104
[161]	train-merror:0.047204	validation-merror:0.103667
[162]	train-merror:0.046759	validation-merror:0.103833
[163]	train-merror:0.046537	validation-merror:0.103833
[164]	train-merror:0.046074	validation-merror:0.103833
[165]	train-merror:0.045926	validation-merror:0.103333
[166]	train-merror:0.045611	validation-merror:0.103167
[167]	train-merror:0.045352	validation-merror:0.103667
[168]	train-merror:0.045111	validation-merror:0.102833
[169]	train-merror:

[303]	train-merror:0.016185	validation-merror:0.097333
[304]	train-merror:0.016185	validation-merror:0.097333
[305]	train-merror:0.015963	validation-merror:0.0975
[306]	train-merror:0.015815	validation-merror:0.098
[307]	train-merror:0.015778	validation-merror:0.097667
[308]	train-merror:0.015667	validation-merror:0.098
[309]	train-merror:0.0155	validation-merror:0.0975
[310]	train-merror:0.015241	validation-merror:0.098
[311]	train-merror:0.015148	validation-merror:0.098167
[312]	train-merror:0.014889	validation-merror:0.098
[313]	train-merror:0.014815	validation-merror:0.097833
[314]	train-merror:0.01487	validation-merror:0.097333
[315]	train-merror:0.014574	validation-merror:0.098
[316]	train-merror:0.014556	validation-merror:0.098
[317]	train-merror:0.014444	validation-merror:0.098
[318]	train-merror:0.014389	validation-merror:0.097833
[319]	train-merror:0.014241	validation-merror:0.0975
[320]	train-merror:0.014296	validation-merror:0.097333
[321]	train-merror:0.014185	validation-m

[455]	train-merror:0.005944	validation-merror:0.0945
[456]	train-merror:0.005889	validation-merror:0.094333
[457]	train-merror:0.00587	validation-merror:0.094333
[458]	train-merror:0.005815	validation-merror:0.094333
[459]	train-merror:0.005759	validation-merror:0.094333
[460]	train-merror:0.005741	validation-merror:0.094333
[461]	train-merror:0.005759	validation-merror:0.094333
[462]	train-merror:0.005759	validation-merror:0.0945
[463]	train-merror:0.005648	validation-merror:0.0945
[464]	train-merror:0.005611	validation-merror:0.0945
[465]	train-merror:0.00563	validation-merror:0.0945
[466]	train-merror:0.005593	validation-merror:0.094
[467]	train-merror:0.005611	validation-merror:0.094
[468]	train-merror:0.005574	validation-merror:0.093667
[469]	train-merror:0.005537	validation-merror:0.094167
[470]	train-merror:0.005463	validation-merror:0.094
[471]	train-merror:0.005537	validation-merror:0.094167
[472]	train-merror:0.005444	validation-merror:0.093667
[473]	train-merror:0.005407	val

### Training the XgBoost classifier on MNIST dataset

In [31]:
param_list = [("eta", 0.08), ("max_depth", 6), ("subsample", 0.8), ("colsample_bytree", 0.8), ("objective", "multi:softmax"), ("eval_metric", "merror"), ("alpha", 8), ("lambda", 2), ("num_class", 10)]
n_rounds = 600
early_stopping = 50
    
d_train = xgb.DMatrix(X_mn_train, label=y_mn_train)
d_val = xgb.DMatrix(X_mn_valid, label=y_mn_valid)
eval_list = [(d_train, "train"), (d_val, "validation")]
bst = xgb.train(param_list, d_train, n_rounds, evals=eval_list, early_stopping_rounds=early_stopping, verbose_eval=True)

[0]	train-merror:0.149333	validation-merror:0.169
Multiple eval metrics have been passed: 'validation-merror' will be used for early stopping.

Will train until validation-merror hasn't improved in 50 rounds.
[1]	train-merror:0.118852	validation-merror:0.138
[2]	train-merror:0.104833	validation-merror:0.123333
[3]	train-merror:0.0945	validation-merror:0.109833
[4]	train-merror:0.090019	validation-merror:0.105333
[5]	train-merror:0.085926	validation-merror:0.099333
[6]	train-merror:0.08337	validation-merror:0.096667
[7]	train-merror:0.081648	validation-merror:0.0965
[8]	train-merror:0.07887	validation-merror:0.0935
[9]	train-merror:0.076611	validation-merror:0.089167
[10]	train-merror:0.074944	validation-merror:0.088667
[11]	train-merror:0.073519	validation-merror:0.087
[12]	train-merror:0.071667	validation-merror:0.085333
[13]	train-merror:0.070056	validation-merror:0.0845
[14]	train-merror:0.068185	validation-merror:0.084
[15]	train-merror:0.066519	validation-merror:0.082333
[16]	trai

[152]	train-merror:0.011833	validation-merror:0.034167
[153]	train-merror:0.011685	validation-merror:0.034
[154]	train-merror:0.011556	validation-merror:0.033833
[155]	train-merror:0.011389	validation-merror:0.033667
[156]	train-merror:0.011296	validation-merror:0.033167
[157]	train-merror:0.011167	validation-merror:0.033333
[158]	train-merror:0.011019	validation-merror:0.033167
[159]	train-merror:0.010944	validation-merror:0.033167
[160]	train-merror:0.010833	validation-merror:0.033
[161]	train-merror:0.010648	validation-merror:0.032667
[162]	train-merror:0.010611	validation-merror:0.032667
[163]	train-merror:0.010611	validation-merror:0.032667
[164]	train-merror:0.010481	validation-merror:0.032667
[165]	train-merror:0.01037	validation-merror:0.0325
[166]	train-merror:0.010241	validation-merror:0.0325
[167]	train-merror:0.010167	validation-merror:0.0325
[168]	train-merror:0.010056	validation-merror:0.032167
[169]	train-merror:0.009963	validation-merror:0.032167
[170]	train-merror:0.00

[304]	train-merror:0.003926	validation-merror:0.028167
[305]	train-merror:0.003889	validation-merror:0.028167
[306]	train-merror:0.00387	validation-merror:0.028167
[307]	train-merror:0.003889	validation-merror:0.028167
[308]	train-merror:0.003907	validation-merror:0.028167
[309]	train-merror:0.00387	validation-merror:0.028167
[310]	train-merror:0.003889	validation-merror:0.028
[311]	train-merror:0.003852	validation-merror:0.027833
[312]	train-merror:0.003852	validation-merror:0.028
[313]	train-merror:0.003815	validation-merror:0.028
[314]	train-merror:0.003796	validation-merror:0.028167
[315]	train-merror:0.003778	validation-merror:0.028
[316]	train-merror:0.003741	validation-merror:0.028
[317]	train-merror:0.003759	validation-merror:0.028
[318]	train-merror:0.003759	validation-merror:0.028
[319]	train-merror:0.003741	validation-merror:0.028
[320]	train-merror:0.003704	validation-merror:0.028
[321]	train-merror:0.003704	validation-merror:0.027833
[322]	train-merror:0.003704	validation-

### Standard scaling the test-sets for both datasets

In [36]:
X_test_std = sc.fit_transform(X_test)

In [32]:
X_test_mn_std = sc.fit_transform(X_test_mnist)

### Predicting with trained classifiers

In [37]:
d_test = xgb.DMatrix(data=X_test_std)
y_pred = bst.predict(d_test)

In [33]:
d_mn_test = xgb.DMatrix(data=X_test_mn_std)
y_mn_pred = bst.predict(d_mn_test)

### Checking accuracy for fashion and MNIST datasets respectively

In [38]:
np.sum(y_pred == y_test) / y_test.shape

array([ 0.8982])

In [34]:
np.sum(y_mn_pred == y_test_mnist) / y_test_mnist.shape

array([ 0.9681])